<h4>Problem </h4>
Create a model to predict the probability of loan-default using Lending Club historical
loan data.

<p>Lending Club enables borrowers to create unsecured personal loans and investors can
search and browse the loans. Investors select loans based on the borrower’s information,
loan amount, loan grade, loan purpose and make money from the interest.
The risk to the investor is if the borrower misses the payments and the loan defaults. If
the risk is predicted and provided to the investor during the selection of the loans then it
helps the investor to make a decision whether to fund the loan.</p>

<p>Data Set : </p>
<p>https://www.lendingclub.com/info/download-data.action </p>
<p>The Lending Club dataset includes detailed information for every loan issued by Lending
Club from 2007 to 2017. The dataset contains a comprehensive list of features that we
can employ to train our model for prediction.</p>

Approach :
1. Data Collection
2. Data Preprocessing
3. Exploratory Data Analysis
4. Modeling
5. Evaluation

In [1]:
import pandas as pd
import numpy as np
import glob

<h4>Data Collection</h4>

In [2]:
# Read files in data folder to a dataframe
path ='data' # use your path
allFiles = glob.glob(path + "/*.csv")
loans_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,skiprows=1,low_memory=False)
    list_.append(df)
loans_df = pd.concat(list_)

In [3]:
loans_df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321864 entries, 0 to 103547
Columns: 151 entries, id to settlement_term
dtypes: float64(112), object(39)
memory usage: 1.5+ GB


There are total of 1321864 rows and 151 columns in the dataset

<h3>Data Wrangling/Preprocessing</h3>

Apply data wrangling techniques to clean and organize the data.

<h4>Now lets see how many null values are present in each column</h4>

In [5]:
loans_df.isnull().sum().sort_values(ascending=False)

sec_app_open_act_il                           1321864
sec_app_earliest_cr_line                      1321864
sec_app_mort_acc                              1321864
sec_app_open_acc                              1321864
sec_app_revol_util                            1321864
sec_app_fico_range_high                       1321864
sec_app_num_rev_accts                         1321864
sec_app_chargeoff_within_12_mths              1321864
sec_app_collections_12_mths_ex_med            1321864
sec_app_mths_since_last_major_derog           1321864
sec_app_fico_range_low                        1321864
sec_app_inq_last_6mths                        1321864
member_id                                     1321864
revol_bal_joint                               1321864
orig_projected_additional_accrued_interest    1317524
hardship_reason                               1316533
hardship_last_payment_amount                  1316533
hardship_payoff_balance_amount                1316533
hardship_length             

From the above you can see there are many missing values in each colum/attribute.
<h4>Now we can get rid of the columns which have more than 30% missing or null values </h4>

In [6]:
#From the above you can see there are many missing values in each colum/attribute.
#Now we can get rid of the columns which have more than 30% missing or null values
threshold_count = len(loans_df)*0.7 
# Drop any column with more than 30% missing values
filtered_loans_df = loans_df.dropna(thresh=threshold_count,axis=1)

In [7]:
filtered_loans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321864 entries, 0 to 103547
Data columns (total 93 columns):
id                            1321864 non-null object
loan_amnt                     1321847 non-null float64
funded_amnt                   1321847 non-null float64
funded_amnt_inv               1321847 non-null float64
term                          1321847 non-null object
int_rate                      1321847 non-null object
installment                   1321847 non-null float64
grade                         1321847 non-null object
sub_grade                     1321847 non-null object
emp_title                     1241887 non-null object
emp_length                    1321847 non-null object
home_ownership                1321847 non-null object
annual_inc                    1321843 non-null float64
verification_status           1321847 non-null object
issue_d                       1321847 non-null object
loan_status                   1321847 non-null object
pymnt_plan         

<h4>After removing the columns that have more than 30% missing columns, we have total number rows of 1321864. but the columns have reduced to 93.</h4>

In [8]:
#Now lets see how many null values are present in each column
filtered_loans_df.isnull().sum().sort_values(ascending=False)

mths_since_recent_inq         182194
num_tl_120dpd_2m              119016
mo_sin_old_il_acct            107552
emp_title                      79977
pct_tl_nvr_dlq                 70446
avg_cur_bal                    70305
mo_sin_old_rev_tl_op           70294
mo_sin_rcnt_rev_tl_op          70294
num_rev_accts                  70294
num_rev_tl_bal_gt_0            70293
num_accts_ever_120_pd          70293
num_actv_bc_tl                 70293
num_actv_rev_tl                70293
num_bc_tl                      70293
num_il_tl                      70293
num_op_rev_tl                  70293
mo_sin_rcnt_tl                 70293
num_tl_30dpd                   70293
num_tl_90g_dpd_24m             70293
num_tl_op_past_12m             70293
tot_hi_cred_lim                70293
total_rev_hi_lim               70293
tot_cur_bal                    70293
total_il_high_credit_limit     70293
tot_coll_amt                   70293
bc_util                        63337
percent_bc_gt_75               63041
b

After filtering the dataset we have 93 columns. Among 93 columns, 67 are of numerical data type and 26 are categorical data types

<h4>Drop the rows which has null values in each column</h4>

In [9]:
filtered_loans_df = filtered_loans_df.drop(['id' ,'url'],axis=1)
#Drop the rows which has null values in each column
filtered_loans_df = filtered_loans_df.dropna(axis=0,how='all')


In [10]:
#Now lets see how many null values are present in each column
filtered_loans_df.isnull().sum().sort_values(ascending=False)

mths_since_recent_inq         182177
num_tl_120dpd_2m              118999
mo_sin_old_il_acct            107535
emp_title                      79960
pct_tl_nvr_dlq                 70429
avg_cur_bal                    70288
mo_sin_rcnt_rev_tl_op          70277
num_rev_accts                  70277
mo_sin_old_rev_tl_op           70277
num_op_rev_tl                  70276
total_rev_hi_lim               70276
tot_cur_bal                    70276
num_actv_bc_tl                 70276
num_actv_rev_tl                70276
tot_coll_amt                   70276
num_bc_tl                      70276
num_il_tl                      70276
mo_sin_rcnt_tl                 70276
num_accts_ever_120_pd          70276
num_rev_tl_bal_gt_0            70276
num_tl_op_past_12m             70276
total_il_high_credit_limit     70276
num_tl_30dpd                   70276
num_tl_90g_dpd_24m             70276
tot_hi_cred_lim                70276
bc_util                        63320
percent_bc_gt_75               63024
b

<h4> Now we will remove the columns that have only one value</h4>

In [11]:

filtered_loans_df = filtered_loans_df.loc[:,filtered_loans_df.apply(pd.Series.nunique) != 1]

In [12]:
filtered_loans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 90 columns):
loan_amnt                     1321847 non-null float64
funded_amnt                   1321847 non-null float64
funded_amnt_inv               1321847 non-null float64
term                          1321847 non-null object
int_rate                      1321847 non-null object
installment                   1321847 non-null float64
grade                         1321847 non-null object
sub_grade                     1321847 non-null object
emp_title                     1241887 non-null object
emp_length                    1321847 non-null object
home_ownership                1321847 non-null object
annual_inc                    1321843 non-null float64
verification_status           1321847 non-null object
issue_d                       1321847 non-null object
loan_status                   1321847 non-null object
pymnt_plan                    1321847 non-null object
purpose            

To handle missing values, we divide the entire datasets into 2 sets
1. Numerical
2. Categorical

In [13]:
# Now we will divide our dataset into 2 dataframes. 
#1. Numerical 2. Categorical
filtered_categorical_loans_df = filtered_loans_df.select_dtypes([object])
filtered_numeric_loans_df = filtered_loans_df.select_dtypes(['float64','int64'])

<h4>Categoical Data</h4>

In [14]:
filtered_categorical_loans_df.head(5)

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
0,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,...,AZ,Jan-1985,83.7%,f,Jan-2015,Dec-2017,Individual,N,Cash,N
1,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,...,GA,Apr-1999,9.4%,f,Apr-2013,Oct-2016,Individual,N,Cash,N
2,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,...,IL,Nov-2001,98.5%,f,Jun-2014,Jun-2017,Individual,N,Cash,N
3,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,...,CA,Feb-1996,21%,f,Jan-2015,Apr-2016,Individual,N,Cash,N
4,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Fully Paid,...,OR,Jan-1996,53.9%,f,Jan-2017,Jan-2017,Individual,N,Cash,N


In [15]:
filtered_categorical_loans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 24 columns):
term                    1321847 non-null object
int_rate                1321847 non-null object
grade                   1321847 non-null object
sub_grade               1321847 non-null object
emp_title               1241887 non-null object
emp_length              1321847 non-null object
home_ownership          1321847 non-null object
verification_status     1321847 non-null object
issue_d                 1321847 non-null object
loan_status             1321847 non-null object
pymnt_plan              1321847 non-null object
purpose                 1321847 non-null object
title                   1298523 non-null object
zip_code                1321847 non-null object
addr_state              1321847 non-null object
earliest_cr_line        1321818 non-null object
revol_util              1321081 non-null object
initial_list_status     1321847 non-null object
last_pymnt_d            

<h4>There are 26 categorical features. For categorical values ,we subsitue the missing values with the highest frequency values of that features.(i.e mode)</h4>

In [16]:
#Now replace the missing values with mode
filtered_categorical_loans_df = filtered_categorical_loans_df.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [17]:
filtered_categorical_loans_df.isnull().sum().sort_values(ascending=False)

debt_settlement_flag    0
disbursement_method     0
int_rate                0
grade                   0
sub_grade               0
emp_title               0
emp_length              0
home_ownership          0
verification_status     0
issue_d                 0
loan_status             0
pymnt_plan              0
purpose                 0
title                   0
zip_code                0
addr_state              0
earliest_cr_line        0
revol_util              0
initial_list_status     0
last_pymnt_d            0
last_credit_pull_d      0
application_type        0
hardship_flag           0
term                    0
dtype: int64

In [18]:
filtered_numeric_loans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 66 columns):
loan_amnt                     1321847 non-null float64
funded_amnt                   1321847 non-null float64
funded_amnt_inv               1321847 non-null float64
installment                   1321847 non-null float64
annual_inc                    1321843 non-null float64
dti                           1321782 non-null float64
delinq_2yrs                   1321818 non-null float64
fico_range_low                1321847 non-null float64
fico_range_high               1321847 non-null float64
inq_last_6mths                1321817 non-null float64
open_acc                      1321818 non-null float64
pub_rec                       1321818 non-null float64
revol_bal                     1321847 non-null float64
total_acc                     1321818 non-null float64
out_prncp                     1321847 non-null float64
out_prncp_inv                 1321847 non-null float64
total_py

<h4>There are a total of 66 numeric featrues in the dataset. To handled missing values for numeric fetures we substitue with the mean value of that feature.</h4>

In [19]:
#Fill the missing values in each column with mean value
filtered_numeric_loans_df = filtered_numeric_loans_df.apply(lambda x: x.fillna(x.mean()),axis=1)


In [20]:
filtered_numeric_loans_df.isnull().sum().sort_values(ascending=False)

total_il_high_credit_limit    0
total_pymnt                   0
total_rev_hi_lim              0
tot_cur_bal                   0
tot_coll_amt                  0
acc_now_delinq                0
collections_12_mths_ex_med    0
last_fico_range_low           0
last_fico_range_high          0
last_pymnt_amnt               0
collection_recovery_fee       0
recoveries                    0
total_rec_late_fee            0
total_rec_int                 0
total_rec_prncp               0
total_pymnt_inv               0
out_prncp_inv                 0
total_bc_limit                0
out_prncp                     0
total_acc                     0
revol_bal                     0
pub_rec                       0
open_acc                      0
inq_last_6mths                0
fico_range_high               0
fico_range_low                0
delinq_2yrs                   0
dti                           0
annual_inc                    0
installment                   0
                             ..
total_ba

In [21]:
preprocessed_data_df = pd.concat([filtered_numeric_loans_df, filtered_categorical_loans_df], axis=1)

<h4>Feature Engineering</h4>

In [22]:
data_dict = pd.ExcelFile("data/LCDataDictionary.xlsx")
# Print the sheet names
print(data_dict.sheet_names)

# Load a sheet into a DataFrame by name: df1
data_dict = data_dict.parse('LoanStats')

['LoanStats', 'browseNotes', 'RejectStats']


In [23]:
data_dict = data_dict.rename(columns={'LoanStatNew': 'name',
                                                 'Description': 'description'})

In [24]:
data_dict.head()

,name,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [25]:
data_dict[data_dict['name'] == 'id']

,name,description
27,id,A unique LC assigned ID for the loan listing.


In [26]:
preprocessed_categorical_data_df = preprocessed_data_df.select_dtypes([object])
preprocessed_numeric_data_df = preprocessed_data_df.select_dtypes(['float64','int64'])

Machine Learning algorithms require the data to be in numeric. So we have to convert the categorical values to numeric.So we will perform the follwing steps 

1. Convert Categorical Columns To Numeric Features
2. Map Ordinal Values To Integers
3. Encode Nominal Values As Dummy Variables

In [27]:
preprocessed_categorical_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 24 columns):
term                    1321847 non-null object
int_rate                1321847 non-null object
grade                   1321847 non-null object
sub_grade               1321847 non-null object
emp_title               1321847 non-null object
emp_length              1321847 non-null object
home_ownership          1321847 non-null object
verification_status     1321847 non-null object
issue_d                 1321847 non-null object
loan_status             1321847 non-null object
pymnt_plan              1321847 non-null object
purpose                 1321847 non-null object
title                   1321847 non-null object
zip_code                1321847 non-null object
addr_state              1321847 non-null object
earliest_cr_line        1321847 non-null object
revol_util              1321847 non-null object
initial_list_status     1321847 non-null object
last_pymnt_d            

In [28]:
data_dict[data_dict['name'] == 'id'] 


,name,description
27,id,A unique LC assigned ID for the loan listing.


In [29]:
data_dict[data_dict['name'] == 'url']

,name,description
111,url,URL for the LC page with listing data.


<h4>The id are url are the information related to LC.so we can drop the two columns.</h4>

In [30]:
#preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['id' ,'url'] , axis=1)

In [31]:
preprocessed_categorical_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 24 columns):
term                    1321847 non-null object
int_rate                1321847 non-null object
grade                   1321847 non-null object
sub_grade               1321847 non-null object
emp_title               1321847 non-null object
emp_length              1321847 non-null object
home_ownership          1321847 non-null object
verification_status     1321847 non-null object
issue_d                 1321847 non-null object
loan_status             1321847 non-null object
pymnt_plan              1321847 non-null object
purpose                 1321847 non-null object
title                   1321847 non-null object
zip_code                1321847 non-null object
addr_state              1321847 non-null object
earliest_cr_line        1321847 non-null object
revol_util              1321847 non-null object
initial_list_status     1321847 non-null object
last_pymnt_d            

In [32]:
#Now filter the columns that have less than 3 unique values
for col in preprocessed_categorical_data_df.columns:
    if (len(preprocessed_categorical_data_df[col].unique()) < 4):
        print(preprocessed_categorical_data_df[col].value_counts())
        print()

 36 months    944664
 60 months    377183
Name: term, dtype: int64

Source Verified    503870
Verified           417390
Not Verified       400587
Name: verification_status, dtype: int64

n    1321249
y        598
Name: pymnt_plan, dtype: int64

w    764274
f    557573
Name: initial_list_status, dtype: int64

Individual    1312547
Joint App        9300
Name: application_type, dtype: int64

N    1321049
Y        798
Name: hardship_flag, dtype: int64

Cash         1318953
DirectPay       2894
Name: disbursement_method, dtype: int64

N    1307295
Y      14552
Name: debt_settlement_flag, dtype: int64



In [33]:
data_dict[data_dict['name'] == 'initial_list_status'].description

29    The initial listing status of the loan. Possib...
Name: description, dtype: object

In [34]:
data_dict[data_dict['name'] == 'pymnt_plan'].description

88    Indicates if a payment plan has been put in pl...
Name: description, dtype: object

There are total 7 columns that have only 2 unique values.

"term" is an important factor /attribute in any loan. 

"debt_settlement_flag" can be important to our analysis.so we don't remove term and debt_settlement_flag from the dataset

"pymnt_plan" and "disbursement_method" and "initial_list_status" are the attributes we get after the loan is isued. so they are not required for the analysis.

"application_type" has two values. Number of Joint App is only 9300 which is very low comapared to Individual application. so we can drop this variable.On the same lines we can drop "hardship_flag".

In [35]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['pymnt_plan' ,'disbursement_method' ,'initial_list_status','application_type','hardship_flag'] , axis=1)

In [36]:
preprocessed_categorical_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 19 columns):
term                    1321847 non-null object
int_rate                1321847 non-null object
grade                   1321847 non-null object
sub_grade               1321847 non-null object
emp_title               1321847 non-null object
emp_length              1321847 non-null object
home_ownership          1321847 non-null object
verification_status     1321847 non-null object
issue_d                 1321847 non-null object
loan_status             1321847 non-null object
purpose                 1321847 non-null object
title                   1321847 non-null object
zip_code                1321847 non-null object
addr_state              1321847 non-null object
earliest_cr_line        1321847 non-null object
revol_util              1321847 non-null object
last_pymnt_d            1321847 non-null object
last_credit_pull_d      1321847 non-null object
debt_settlement_flag    

In [37]:
data_dict[data_dict['name'] == 'issue_d'].description


35    The month which the loan was funded
Name: description, dtype: object

In [38]:
data_dict[data_dict['name'] == 'last_pymnt_d'].description

40    Last month payment was received
Name: description, dtype: object

issue_d is th month the loan was funded and last_pymnt_d is the last month payment. As these values are obtained after the loan is issued. so we can drop these variables.

In [39]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['issue_d' ,'last_pymnt_d'] , axis=1)

In [40]:
preprocessed_categorical_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 17 columns):
term                    1321847 non-null object
int_rate                1321847 non-null object
grade                   1321847 non-null object
sub_grade               1321847 non-null object
emp_title               1321847 non-null object
emp_length              1321847 non-null object
home_ownership          1321847 non-null object
verification_status     1321847 non-null object
loan_status             1321847 non-null object
purpose                 1321847 non-null object
title                   1321847 non-null object
zip_code                1321847 non-null object
addr_state              1321847 non-null object
earliest_cr_line        1321847 non-null object
revol_util              1321847 non-null object
last_credit_pull_d      1321847 non-null object
debt_settlement_flag    1321847 non-null object
dtypes: object(17)
memory usage: 181.5+ MB


In [41]:
# Print unique values for purpose and title
preprocessed_categorical_data_df.purpose.value_counts()
preprocessed_categorical_data_df.title.value_counts()

Debt consolidation                          671894
Credit card refinancing                     250516
Home improvement                             69997
Other                                        59637
Major purchase                               22082
Debt Consolidation                           15763
Medical expenses                             11882
Business                                     11280
Car financing                                10186
Moving and relocation                         7042
Vacation                                      6916
Consolidation                                 5385
debt consolidation                            4837
Home buying                                   4287
Debt Consolidation Loan                       3804
Credit Card Consolidation                     2360
consolidation                                 2175
Personal Loan                                 2133
Consolidation Loan                            1778
Home Improvement               

From the above you can see the title and purpose have similar values.
<p>'title' is the description of the purpose of the loan. Purpose has fewer values.So we can drop the tilte.</p>

<p>Similarly, zip_code and addr_state gives the same infomration. so we can drop zip_code</p>

<p>Also, subgrade is the sub category of grade.So we will drop subgrade column too

In [42]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['title','zip_code','sub_grade'],axis=1)

In [43]:
preprocessed_categorical_data_df.head()

,term,int_rate,grade,emp_title,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,earliest_cr_line,revol_util,last_credit_pull_d,debt_settlement_flag
0,36 months,10.65%,B,Teacher,10+ years,RENT,Verified,Fully Paid,credit_card,AZ,Jan-1985,83.7%,Dec-2017,N
1,60 months,15.27%,C,Ryder,< 1 year,RENT,Source Verified,Charged Off,car,GA,Apr-1999,9.4%,Oct-2016,N
2,36 months,15.96%,C,Teacher,10+ years,RENT,Not Verified,Fully Paid,small_business,IL,Nov-2001,98.5%,Jun-2017,N
3,36 months,13.49%,C,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Fully Paid,other,CA,Feb-1996,21%,Apr-2016,N
4,60 months,12.69%,B,University Medical Group,1 year,RENT,Source Verified,Fully Paid,other,OR,Jan-1996,53.9%,Jan-2017,N


<h4>The colums int_rate and revol_util are in percantage. we will convert it to numeric values./h4>

In [44]:
#The colums int_rate and revol_util are in percantage. we will convert it to numeric values.
preprocessed_categorical_data_df['int_rate'] = preprocessed_categorical_data_df['int_rate'].replace('%','',regex=True).astype('float')

preprocessed_categorical_data_df['revol_util'] = preprocessed_categorical_data_df['revol_util'].replace('%','',regex=True).astype('float')

In [45]:
preprocessed_categorical_data_df['term'].value_counts()

 36 months    944664
 60 months    377183
Name: term, dtype: int64

<h4>Convert the term column to numeruc by removing the months.</h4>

In [46]:
#Convert the term column to numeruc by removing the months.
preprocessed_categorical_data_df['term'] = preprocessed_categorical_data_df['term'].replace('months','',regex=True).astype('int')


In [47]:
preprocessed_categorical_data_df.emp_title.value_counts()

Teacher                                  101680
Manager                                   18643
Owner                                     10383
Registered Nurse                           8906
RN                                         8638
Supervisor                                 8200
Sales                                      7108
Driver                                     6829
Project Manager                            6346
Office Manager                             5530
General Manager                            5246
Director                                   5104
owner                                      4837
manager                                    4526
President                                  4447
teacher                                    4276
Engineer                                   4262
Vice President                             3636
driver                                     3501
Operations Manager                         3396
Attorney                                

There are total of 381900 unique values for emp_title. The dimension of data increases when this variable is one-hot encoded. So we can drop emp_title for our analysis

In [48]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['emp_title'],axis=1)

In [49]:
preprocessed_categorical_data_df.head()

,term,int_rate,grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,earliest_cr_line,revol_util,last_credit_pull_d,debt_settlement_flag
0,36,10.65,B,10+ years,RENT,Verified,Fully Paid,credit_card,AZ,Jan-1985,83.7,Dec-2017,N
1,60,15.27,C,< 1 year,RENT,Source Verified,Charged Off,car,GA,Apr-1999,9.4,Oct-2016,N
2,36,15.96,C,10+ years,RENT,Not Verified,Fully Paid,small_business,IL,Nov-2001,98.5,Jun-2017,N
3,36,13.49,C,10+ years,RENT,Source Verified,Fully Paid,other,CA,Feb-1996,21.0,Apr-2016,N
4,60,12.69,B,1 year,RENT,Source Verified,Fully Paid,other,OR,Jan-1996,53.9,Jan-2017,N


In [50]:
data_dict[data_dict['name'] == 'earliest_cr_line'].description

18    The month the borrower's earliest reported cre...
Name: description, dtype: object

In [51]:
data_dict[data_dict['name'] == 'last_credit_pull_d'].description

36    The most recent month LC pulled credit for thi...
Name: description, dtype: object

'earliest_cr_line and last_credit_pull_d columns contain date values. To use these variables it would need good amount of feature engineering. so we drop these columns.

In [52]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['earliest_cr_line','last_credit_pull_d'],axis=1)

In [53]:
preprocessed_categorical_data_df.head()

,term,int_rate,grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,revol_util,debt_settlement_flag
0,36,10.65,B,10+ years,RENT,Verified,Fully Paid,credit_card,AZ,83.7,N
1,60,15.27,C,< 1 year,RENT,Source Verified,Charged Off,car,GA,9.4,N
2,36,15.96,C,10+ years,RENT,Not Verified,Fully Paid,small_business,IL,98.5,N
3,36,13.49,C,10+ years,RENT,Source Verified,Fully Paid,other,CA,21.0,N
4,60,12.69,B,1 year,RENT,Source Verified,Fully Paid,other,OR,53.9,N


In [54]:
preprocessed_categorical_data_df['emp_length'].value_counts()

10+ years    441566
2 years      118474
3 years      104762
< 1 year     102526
1 year        86251
5 years       82184
4 years       78350
n/a           73049
8 years       62731
6 years       61608
7 years       58802
9 years       51544
Name: emp_length, dtype: int64

In [55]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0

    }
}

In [56]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.replace(mapping_dict)

In [57]:
preprocessed_categorical_data_df.head()

,term,int_rate,grade,emp_length,home_ownership,verification_status,loan_status,purpose,addr_state,revol_util,debt_settlement_flag
0,36,10.65,B,10,RENT,Verified,Fully Paid,credit_card,AZ,83.7,N
1,60,15.27,C,0,RENT,Source Verified,Charged Off,car,GA,9.4,N
2,36,15.96,C,10,RENT,Not Verified,Fully Paid,small_business,IL,98.5,N
3,36,13.49,C,10,RENT,Source Verified,Fully Paid,other,CA,21.0,N
4,60,12.69,B,1,RENT,Source Verified,Fully Paid,other,OR,53.9,N


In [58]:
preprocessed_categorical_data_df.addr_state.value_counts()

CA    187415
NY    109594
TX    108175
FL     92668
IL     53192
NJ     49149
PA     45895
OH     44631
GA     43547
VA     37944
NC     37003
MI     34590
MD     31104
AZ     30874
MA     30424
WA     27736
CO     27605
MN     23656
IN     21449
MO     21159
CT     20326
TN     20160
NV     18609
WI     17366
AL     16533
SC     16132
LA     15573
OR     15563
KY     12775
OK     12147
KS     11494
AR      9975
UT      9061
NM      7227
MS      6686
HI      6589
NH      6391
RI      5775
WV      5156
DE      3763
MT      3731
DC      3391
NE      3292
AK      3211
WY      2963
VT      2739
SD      2694
ME      1934
ND      1530
ID      1237
IA        14
Name: addr_state, dtype: int64

There are almost 50 different states. so it is going to increase the dimensions of the datasets when converted to numeric value. So to decrease the complexity we can drop addr_state from the dataset.

In [59]:
preprocessed_categorical_data_df = preprocessed_categorical_data_df.drop(['addr_state'],axis=1)

In [60]:
preprocessed_categorical_data_df.head()

,term,int_rate,grade,emp_length,home_ownership,verification_status,loan_status,purpose,revol_util,debt_settlement_flag
0,36,10.65,B,10,RENT,Verified,Fully Paid,credit_card,83.7,N
1,60,15.27,C,0,RENT,Source Verified,Charged Off,car,9.4,N
2,36,15.96,C,10,RENT,Not Verified,Fully Paid,small_business,98.5,N
3,36,13.49,C,10,RENT,Source Verified,Fully Paid,other,21.0,N
4,60,12.69,B,1,RENT,Source Verified,Fully Paid,other,53.9,N


In [61]:
data_dict[data_dict['name'] == 'loan_status'].description

42    Current status of the loan
Name: description, dtype: object

In [62]:
preprocessed_categorical_data_df['loan_status'].value_counts()

Fully Paid                                             639250
Current                                                479981
Charged Off                                            170927
Late (31-120 days)                                      16750
In Grace Period                                          8304
Late (16-30 days)                                        3773
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                   113
Name: loan_status, dtype: int64

In [63]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
preprocessed_categorical_data_df['loan_status'] = le.fit_transform(preprocessed_categorical_data_df['loan_status'])

In [64]:
preprocessed_categorical_data_df = pd.get_dummies(preprocessed_categorical_data_df,drop_first=True)

In [65]:
preprocessed_categorical_data_df.head()

,term,int_rate,emp_length,loan_status,revol_util,grade_B,grade_C,grade_D,grade_E,grade_F,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,debt_settlement_flag_Y
0,36,10.65,10,5,83.7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60,15.27,0,0,9.4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,36,15.96,10,5,98.5,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,36,13.49,10,5,21.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,60,12.69,1,5,53.9,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


<h4>Now lets explore the numeric values</h4>

In [67]:
preprocessed_numeric_data_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,5000.0,5000.0,4975.0,162.87,24000.0,27.65,0.0,735.0,739.0,1.0,...,2297.071725,2297.071725,2297.071725,2297.071725,0.0,0.0,2297.071725,2297.071725,2297.071725,2297.071725
1,2500.0,2500.0,2500.0,59.83,30000.0,1.00,0.0,740.0,744.0,5.0,...,1387.724688,1387.724688,1387.724688,1387.724688,0.0,0.0,1387.724688,1387.724688,1387.724688,1387.724688
2,2400.0,2400.0,2400.0,84.33,12252.0,8.72,0.0,735.0,739.0,2.0,...,1097.811464,1097.811464,1097.811464,1097.811464,0.0,0.0,1097.811464,1097.811464,1097.811464,1097.811464
3,10000.0,10000.0,10000.0,339.31,49200.0,20.00,0.0,690.0,694.0,1.0,...,3901.451875,3901.451875,3901.451875,3901.451875,0.0,0.0,3901.451875,3901.451875,3901.451875,3901.451875
4,3000.0,3000.0,3000.0,67.79,80000.0,17.94,0.0,695.0,699.0,0.0,...,4123.992443,4123.992443,4123.992443,4123.992443,0.0,0.0,4123.992443,4123.992443,4123.992443,4123.992443


In [68]:
preprocessed_numeric_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 66 columns):
loan_amnt                     1321847 non-null float64
funded_amnt                   1321847 non-null float64
funded_amnt_inv               1321847 non-null float64
installment                   1321847 non-null float64
annual_inc                    1321847 non-null float64
dti                           1321847 non-null float64
delinq_2yrs                   1321847 non-null float64
fico_range_low                1321847 non-null float64
fico_range_high               1321847 non-null float64
inq_last_6mths                1321847 non-null float64
open_acc                      1321847 non-null float64
pub_rec                       1321847 non-null float64
revol_bal                     1321847 non-null float64
total_acc                     1321847 non-null float64
out_prncp                     1321847 non-null float64
out_prncp_inv                 1321847 non-null float64
total_py

Drop the featueres which are not useful for the analysis as they are obtained after the loan is issued.

1. funded_amnt
2. funded_amnt_inv
3. out_prncp - Remaining outstanding principal for total amount funded
4. out_prncp_inv - Remaining outstanding principal for portion of total amount funded by investors
5. total_pymnt
6. total_pymnt_inv
7. total_rec_prncp - Principal received to date
8. total_rec_int - Interest received to date
9. total_rec_late_fee - Late fees received to date
10. last_pymnt_amnt

In [69]:
preprocessed_numeric_data_df = preprocessed_numeric_data_df.drop(['funded_amnt','funded_amnt_inv','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','last_pymnt_amnt'],axis=1)

In [70]:
preprocessed_numeric_data_df = preprocessed_numeric_data_df.drop(['last_fico_range_high','last_fico_range_low'],axis=1)

In [71]:
filtered_numeric_loans_df['fico_average'] = (filtered_numeric_loans_df['fico_range_high'] + filtered_numeric_loans_df['fico_range_low']) / 2

In [72]:
preprocessed_numeric_data_df = preprocessed_numeric_data_df.drop(['fico_range_high','fico_range_low'],axis=1)

In [73]:
preprocessed_numeric_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 52 columns):
loan_amnt                     1321847 non-null float64
installment                   1321847 non-null float64
annual_inc                    1321847 non-null float64
dti                           1321847 non-null float64
delinq_2yrs                   1321847 non-null float64
inq_last_6mths                1321847 non-null float64
open_acc                      1321847 non-null float64
pub_rec                       1321847 non-null float64
revol_bal                     1321847 non-null float64
total_acc                     1321847 non-null float64
recoveries                    1321847 non-null float64
collection_recovery_fee       1321847 non-null float64
collections_12_mths_ex_med    1321847 non-null float64
acc_now_delinq                1321847 non-null float64
tot_coll_amt                  1321847 non-null float64
tot_cur_bal                   1321847 non-null float64
total_re

In [74]:
final_filtered_df = pd.concat([preprocessed_numeric_data_df, preprocessed_categorical_data_df], axis=1)

In [75]:
final_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 84 columns):
loan_amnt                              1321847 non-null float64
installment                            1321847 non-null float64
annual_inc                             1321847 non-null float64
dti                                    1321847 non-null float64
delinq_2yrs                            1321847 non-null float64
inq_last_6mths                         1321847 non-null float64
open_acc                               1321847 non-null float64
pub_rec                                1321847 non-null float64
revol_bal                              1321847 non-null float64
total_acc                              1321847 non-null float64
recoveries                             1321847 non-null float64
collection_recovery_fee                1321847 non-null float64
collections_12_mths_ex_med             1321847 non-null float64
acc_now_delinq                         1321847 non-null 

In [76]:
final_filtered_df = final_filtered_df.dropna(axis=0,how='all')

In [77]:
final_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321847 entries, 0 to 103545
Data columns (total 84 columns):
loan_amnt                              1321847 non-null float64
installment                            1321847 non-null float64
annual_inc                             1321847 non-null float64
dti                                    1321847 non-null float64
delinq_2yrs                            1321847 non-null float64
inq_last_6mths                         1321847 non-null float64
open_acc                               1321847 non-null float64
pub_rec                                1321847 non-null float64
revol_bal                              1321847 non-null float64
total_acc                              1321847 non-null float64
recoveries                             1321847 non-null float64
collection_recovery_fee                1321847 non-null float64
collections_12_mths_ex_med             1321847 non-null float64
acc_now_delinq                         1321847 non-null 

In [78]:
final_filtered_df.isnull().sum().sort_values(ascending=False)

debt_settlement_flag_Y        0
num_accts_ever_120_pd         0
delinq_amnt                   0
mo_sin_old_il_acct            0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
mort_acc                      0
mths_since_recent_bc          0
mths_since_recent_inq         0
num_actv_bc_tl                0
purpose_wedding               0
num_actv_rev_tl               0
num_bc_sats                   0
num_bc_tl                     0
num_il_tl                     0
num_op_rev_tl                 0
num_rev_accts                 0
num_rev_tl_bal_gt_0           0
num_sats                      0
chargeoff_within_12_mths      0
bc_util                       0
bc_open_to_buy                0
avg_cur_bal                   0
installment                   0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
                             ..
purpose_

In [79]:
final_filtered_df.to_csv('final_filtered.csv')